In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-239222
Azure region: southcentralus
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-239222


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AT7ZWW5XC to authenticate.


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = "udacity-project"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
cluster = ComputeTarget.create(ws, cluster_name, config)

cluster

AmlCompute(workspace=Workspace.create(name='quick-starts-ws-239222', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-239222'), name=udacity-project, id=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourceGroups/aml-quickstarts-239222/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-239222/computes/udacity-project, type=AmlCompute, provisioning_state=Creating, location=southcentralus, tags=None)

In [26]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({
        'C': choice(0.5, 1., 5., 10., 25., 35., 45., 50., 55., 60., 75., 100.),
        'max_iter': choice(50, 100, 200, 500, 1000)
    }

)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.', script='train.py',
                      arguments=['--C', 1.0, '--max_iter', 100],
                      compute_target=cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=50,
                                     max_concurrent_runs=4
                    )

In [27]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

In [28]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_4ef24334-c2ca-446b-a73c-be8314716e85
Web View: https://ml.azure.com/runs/HD_4ef24334-c2ca-446b-a73c-be8314716e85?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-239222/workspaces/quick-starts-ws-239222&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-07-25T11:11:10.567846][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-07-25T11:11:11.0906309Z][SCHEDULER][INFO]Scheduling job, id='HD_4ef24334-c2ca-446b-a73c-be8314716e85_0' 
[2023-07-25T11:11:11.2103938Z][SCHEDULER][INFO]Scheduling job, id='HD_4ef24334-c2ca-446b-a73c-be8314716e85_1' 
[2023-07-25T11:11:11.2884227Z][SCHEDULER][INFO]Scheduling job, id='HD_4ef24334-c2ca-446b-a73c-be8314716e85_2' 
[2023-07-25T11:11:11.4017781Z][SCHEDULER][INFO]Scheduling job, id='HD_4ef24334-c2ca-446b-a73c-be8314716e85_3' 
[2023-07-25T11:11:11.359717][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_4ef24334-c2ca-446b-a73c-be8314716e85',
 'target': 'udacity-project',
 'status': 'Completed',
 'startTimeUtc': '2023-07-25T11:11:09.770046Z',
 'endTimeUtc': '2023-07-25T11:27:16.410149Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'f701dd19-eac7-4513-b822-97317956b5bf',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '60',
  'score': '0.9082301529497451',
  'best_child_run_id': 'HD_4ef24334-c2ca-446b-a73c-be8314716e85_36',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_4ef24334-c2ca-446b-a73c-be8314716e85_36'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlC

In [46]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print(f'Best Run id: {best_run.id}')
print(f'Accuracy: {best_run_metrics["Accuracy"]}')
print(f'C: {parameter_values[0]}')
print(f'max_iter: {parameter_values[1]}')

# joblib.dump(value=model??, filename='outputs/hyperdrive_model.pkl')

Best Run id: HD_4ef24334-c2ca-446b-a73c-be8314716e85_36
Accuracy: 0.9082301529497451
C: --C
max_iter: 1


In [47]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [83]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [92]:
import pandas as pd
from azureml.core import Dataset, Datastore

df_train = pd.concat([x,y], axis=1)

datastore = Datastore.get(ws, 'workspaceblobstore')
train_ds = Dataset.Tabular.register_pandas_dataframe(df_train, datastore, "train set", show_progress=True)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/cc38f557-9e2e-47e0-a39d-ff6f7dc41295/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [94]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    iterations=30,
    iteration_timeout_minutes=5,
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='y',
    compute_target=cluster,
    n_cross_validations=5)

In [89]:
# Submit your automl run

automl_run = exp.submit(config=automl_config, show_output=True)

ConfigException: ConfigException:
	Message: Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]Please refer to documentation for converting to Supported types: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type '<class 'pandas.core.frame.DataFrame'>' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset]Please refer to documentation for converting to Supported types: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.dataset.dataset?view=azure-ml-py",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_er

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###